In [1]:
import os
import sys
sys.path.append('/scratch/users/k21066795/prj_normal/RandStainNA')
from randstainna import RandStainNA

import glob
import h5py
import random
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from randstainna import RandStainNA

import cv2
from utils_data import add_ageGroup
from utils_features import ensemble_feas_labels, labelingFeature, CSVDataset_WSI, get_model, extract_features, sampeled_UMP
from utils import plot_multiple, plot_oneline


def ROIs_per_mm2(wsiname, WSIs, WSI_QC, rootdir):
    '''
    given a wsi, calculate the tissue area, number of ROIs detected per mm^2, and percentage of epithelium/tissue area (%)
    '''
    folder = os.path.join(WSI_QC, wsiname+'.ndpi')
    mask_pt = [i for i in os.listdir(folder) if '_mask_use.npy' in i]
    mask_pt = os.path.join(WSI_QC, wsiname+'.ndpi', mask_pt[0])
    mask = np.load(mask_pt)
    
    wsi_pt = os.path.join(WSIs, wsiname+".ndpi")
    wsi = openslide.OpenSlide(wsi_pt)
    scale_factor = wsi.level_dimensions[0][1] // mask.shape[0]
    print(f'scale_factor: {scale_factor}')
    mpp = wsi.properties['openslide.mpp-y'] # um/pixel
    wsi_area = np.sum(mask) * scale_factor * scale_factor * float(mpp)*0.001 * float(mpp) * 0.001 # mm^2
    
    print(f'wsi_area: {wsi_area}')

    csv_folder = os.path.join(rootdir, wsiname)
    csv_pt = glob.glob(f"{csv_folder}/*_patch_merge.csv")[0]
    csv_df = pd.read_csv(csv_pt)
    patient_age = np.unique(csv_df['age'])[0]
    patient_id = np.unique(csv_df['patient_id'])[0]
    wsi_id = np.unique(csv_df['wsi_id'])[0]
    
    print(f'patient_age: {patient_age}')

    ROI_num = len(np.unique(csv_df['roi_id']))
    ROI_adjust = np.around(ROI_num/wsi_area, 3)
    print(f'ROI_count_adjust: {ROI_adjust} lobules per mm^2')
    

    epi_num = np.sum(csv_df['cls']=='epithelium')
    epi_area_mm2 = epi_num * 128 * 128 *0.001 *0.001
    epi_percent = np.around(epi_area_mm2/wsi_area, 3) # percentage of epithelium on the slide
    
    print(f'epi_count_adjust: {epi_adjust} epithelium patches per mm^2')
      
    return patient_id, wsi_id, ROI_adjust, epi_percent, patient_age


/scratch/users/k21066795/conda/torch_env/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
/scratch/users/k21066795/conda/torch_env/lib/python3.9/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)


In [ ]:
# paths
WSIs = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/WSIs/KHP_RM'  # where to get WSIs???
WSI_QC = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/WSI_QC/KHP_RM'  # where to get WSIs???
rootdir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/RootDir/KHP_RM' # where to get _patch.csv, .h5 files
clinic = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/DATA' # where to get clinic.csv
ResultDir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/Results/'

#--------------------------check clinic----------------------------#
check distribution of patients in three age groups
usual-risk: middle highest
high-risk: ?? highest

density plot, age moves to left

In [ ]:
# get KHP_RM df
KHP_pt = glob.glob(f"{clinic}/*KHP*")[0]
KHP_RM = pd.read_csv(KHP_pt)
KHP_RM = add_ageGroup(KHP_RM) # add categorical age group based on patients' age
KHP_RM = KHP_RM.loc[KHP_RM['source']=='RM'] # only get RM patients
_, counts = np.unique(KHP_RM['age_group'], return_counts=True)
m, y, o = counts
print(f"percentage of patients in each group \n <30y: {y}  30-50y: {m}  >50y {o}")


In [ ]:
# get KHP_RRM df
KHP_pt = glob.glob(f"{clinic}/*KHP*")[0]
KHP_RRM = pd.read_csv(KHP_pt)
KHP_RRM = add_ageGroup(KHP_RRM) # add categorical age group based on patients' age
KHP_RRM = KHP_RRM.loc[KHP_RRM['source']=='RRM'] # only get RM patients
_, counts = np.unique(KHP_RRM['age_group'], return_counts=True)
m, y, o = counts
print(f"percentage of patients in each group \n <30y: {y}  30-50y: {m}  >50y {o}")

In [ ]:
# get SGK_healthy
SGK_healthy_pt = glob.glob(f"{clinic}/*SGK*")[0]
SGK_healthy = pd.read_csv(SGK_healthy_pt)
SGK_healthy = add_ageGroup(SGK_healthy) # add categorical age group based on patients' age
# SGK_healthy = SGK_healthy.loc[SGK_healthy['source']=='RM'] # only get RM patients
print(np.unique(SGK_healthy['age_group'], return_counts=True)) # check patient's age group distribution

In [ ]:
# get BCI
BCI = pd.read_csv(f'{clinic}/BCI_clinic.csv')
BCI = add_ageGroup(BCI)
_, counts = np.unique(BCI['age_group'], return_counts=True)
m, y, o = counts
print(f"percentage of patients in each group \n <30y: {y}  30-50y: {m}  >50y {o}")

In [ ]:
BCI

In [ ]:
BCI.groupby(['source', 'age_group'])['patient_id'].count()

BCI['source2'] = BCI['source']

BCI.loc[BCI['source2'] == 'Ipsilateral', 'source2'] = 'Cancer'


# BCI.groupby(['source', 'age_group'])['patient_id'].count()
BCI

#---------------check distributions of ROIsCount_mm2, epitheliumArea_mm2---------------------#

In [ ]:
rootdir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/RootDir/SGK_healthy' # where to get _patch.csv, .h5 files
slide_dir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/WSI_QC/SGK_healthy'


csv_files = []
TC_files = []
h5_files = []
for wsiname in os.listdir(rootdir):
    wsiname = wsiname.split('.svs')[0]
    folder = os.path.join(rootdir, wsiname)
    
    csv_pt = glob.glob(f"{folder}/*_patch.csv")
    if len(csv_pt)>0:
        csv_files.append(wsiname)

    TC_pt = glob.glob(f"{folder}/*_TCprobmask.npy")
    if len(TC_pt)>0:
        TC_files.append(TC_pt[0])

    h5_pt = glob.glob(f"{folder}/*.h5")
    if len(h5_pt)>0:
        h5_files.append(h5_pt[0])

print(len(TC_files))
print(len(csv_files))
print(len(h5_files))

In [ ]:
rootdir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/RootDir/SGK_healthy'

wsi_noTC = []
for wsiname in os.listdir(rootdir):
    wsiname = wsiname.split('.svs')[0]
    folder = os.path.join(rootdir, wsiname)
    TC_pt = glob.glob(f"{folder}/*_TCprobmask.npy")
    if len(TC_pt) == 0:
        wsi_noTC.append(wsiname)
    

In [ ]:
wsinames = []
for wsiname in os.listdir(rootdir):
    csv_folder = os.path.join(rootdir, wsiname.split('.ndpi')[0])
    csv_pt = glob.glob(f"{csv_folder}/*_patch_merge.csv")
    if len(csv_pt)>0:
        wsinames.append(wsiname)


roiDict = {'patient_id':[], 'wsi_id':[], 'ROIs_mm2': [], 'epi_percent':[], 'patient_ages': []}
for wsiname in wsinames:
    patient_id, wsi_id, ROI_adjust, epi_percent, patient_age = ROIs_per_mm2(wsiname, WSIs, WSI_QC, rootdir)
    roiDict['patient_id'].append(patient_id)
    roiDict['wsi_id'].append(wsi_id)
    roiDict['ROIs_mm2'].append(ROI_adjust)
    roiDict['epi_percent'].append(epi_percent)
    roiDict['patient_ages'].append(patient_age)

In [ ]:
import pandas as pd
roiDict_df = pd.DataFrame.from_dict(roiDict)
sorted_df = roiDict_df.sort_values(by='patient_ages')

roiDict_mean_df = sorted_df[['patient_id','ROIs_mm2','epi_percent','patient_ages']].groupby('patient_id').mean().reset_index()
roiDict_mean_df = roiDict_mean_df.sort_values(by='patient_ages')
roiDict_mean_df['patient_id'] = roiDict_mean_df['patient_id'].astype(str)
roiDict_mean_df.to_csv(f"{ResultDir}/Tables/ROI_age_relation_KHP_RM.csv", index=False)

In [ ]:
plt.bar(roiDict_mean_df['patient_id'], roiDict_mean_df['ROIs_mm2'])

In [ ]:
plt.bar(roiDict_mean_df['patient_id'], roiDict_mean_df['epi_percent'])


In [ ]:
plt.bar(roiDict_mean_df['patient_id'], roiDict_mean_df['patient_ages'])


#--------------------------check mask----------------------------#

In [ ]:
wsiname = os.listdir(WSI_QC)[2]
folder = os.path.join(WSI_QC, wsiname)
mask_pt = [i for i in os.listdir(folder) if '_mask_use.npy' in i]
mask_pt = os.path.join(WSI_QC, wsiname, mask_pt[0])
mask = np.load(mask_pt)

wsi_pt = os.path.join(WSIs, wsiname)
wsi = openslide.OpenSlide(wsi_pt)
mpp = wsi.properties['openslide.mpp-y'] # um/pixel

np.sum(mask) * float(mpp)*0.001 * float(mpp) * 0.001 # 0.15 / mm^2

In [ ]:
num_adjust = ROIs_per_mm2(wsiname, WSIs, WSI_QC, rootdir)

In [ ]:
im = wsi.read_region((0,0), 5, wsi.level_dimensions[5]).convert('RGB')
plt.imshow(mask)

#--------------------------check lobule types----------------------------#

In [ ]:
# visualise lobule types on WSIs
import cv2
import json
import openslide

def generate_mask(wsi_path, anno_pt, scale_factor):
    slide=openslide.OpenSlide(wsi_path)
    with open(anno_pt, "r") as f:
        shapes = json.load(f)
    
    # level = slide.level_count - 1
    # scale_factor = 1/slide.level_downsamples[level]
    # width, height = slide.level_dimensions[level]

    width, height = slide.level_dimensions[0]
    width, height = int(width * scale_factor), int(height * scale_factor)
    
    background = np.zeros((height, width, 3), np.uint8)
    mask = np.full((height, width, 3), background, dtype=np.uint8)
    
    for shape in shapes["features"]:
        points = shape["geometry"]["coordinates"][0]
        points = np.array([(p[0], p[1]) for p in points])
        points = points * scale_factor
        points = points.astype(int)
    
        cls = shape["properties"]["classification"]["name"]
        if cls == "3":
          color = (255, 0, 0)
          mask = cv2.drawContours(mask, [points], -1, color=color, thickness=-1)
          mask = cv2.fillPoly(mask, [points], color=color)
        elif cls == "2":
          color = (0, 255, 0)
          mask = cv2.drawContours(mask, [points], -1, color=color, thickness=-1)
          mask = cv2.fillPoly(mask, [points], color=color)
        elif cls == "1":
          color = (0, 0, 255)
          mask = cv2.drawContours(mask, [points], -1, color=color, thickness=-1)
          mask = cv2.fillPoly(mask, [points], color=color)
        
    return mask
    

In [ ]:
# get wsi_ids
annoDir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/Manual_anno/KHP_lobule'
wsi_ids = [i.split('.geojson')[0] for i in os.listdir(annoDir)]
wsi_pts = [glob.glob(f"/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/WSIs/KHP_RM/{wsiname}*.*")[0] for wsiname in wsi_ids]
wsi_pts.sort()
wsi_pts

In [ ]:
# get annotations
anno_pts = [os.path.join(annoDir, i) for i in os.listdir(annoDir)]
anno_pts.sort()
anno_pts

In [ ]:
# get patch ids
csvDir = "/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/RootDir/KHP_RM/"
wsi_ids = [i.split('.geojson')[0] for i in os.listdir(annoDir)]
csv_pts = [f"{csvDir}{wsi_id}/{wsi_id}_patch_merge.csv" for wsi_id in wsi_ids]
csv_pts.sort()
csv_pts

In [ ]:
index = 15

wsi_path = wsi_pts[index]
anno_pt = anno_pts[index]
csv_pt = csv_pts[index]
print(wsi_path)
print(anno_pt)
print(csv_pt)

In [ ]:
def get_lobule_patches(patch_ids):
    type1 = []
    type2 = []
    type3 = []
    for patch_id in patch_ids:
        grid_x, grid_y = patch_id.split("_")[-3], patch_id.split("_")[-2]
        grid_x, grid_y = int(grid_x), int(grid_y)
        value = tuple(mask_im[grid_y, grid_x, :])
        if value == (255, 0, 0):
            lobule = 'type3'
            type3.append(patch_id)
            
        elif value == (0, 255, 0):
            lobule = 'type2'
            type2.append(patch_id)
        
        elif value == (0, 0, 255):
            lobule = 'type1'
            type1.append(patch_id)
    return type1, type2, type3


def parse_patch_id(patch_id):
    wsiname = "_".join(patch_id.split("_")[:3])
    ROI_id = "_".join(patch_id.split("_")[4:8])
    gridx, gridy, patch_size = patch_id.split("_")[-3:]
    gridx, gridy, patch_size = int(gridx), int(gridy), int(patch_size)
    
    return wsiname, ROI_id, gridx, gridy, patch_size

In [ ]:
df = pd.read_csv(csv_pt)
print(f"patient age: {np.unique(df['age'])}")

patch_ids = [i for i in df['patch_id']]
type1, type2, type3 = get_lobule_patches(patch_ids)
print(f"type1: {len(type1)} type2: {len(type2)} type3: {len(type3)}")

together = len(type1) + len(type2) + len(type3)
print(f"type1: {len(type1)/together} type2: {len(type2)/together} type3: {len(type3)/together}")

In [ ]:
index = random.sample(range(0, len(type3)), 12)
imgs = []
for i in index:
    patch_id = type3[i]
    wsiname, ROI_id, gridx, gridy, patch_size = parse_patch_id(patch_id)
    im = wsi.read_region((int(gridx*patch_size), int(gridy*patch_size)), 0, (patch_size, patch_size)).convert('RGB')
    imgs.append(im)

plot_multiple(imgs, ['type3']*12, 3,4, (10,5), 'tab20')

In [ ]:
index = random.sample(range(0, len(type2)), 12)
imgs = []
for i in index:
    patch_id = type2[i]
    wsiname, ROI_id, gridx, gridy, patch_size = parse_patch_id(patch_id)
    im = wsi.read_region((int(gridx*patch_size), int(gridy*patch_size)), 0, (patch_size, patch_size)).convert('RGB')
    imgs.append(im)

plot_multiple(imgs, ['type2']*12, 3,4, (10, 5), 'tab20')

In [ ]:
index = random.sample(range(0, len(type1)), 12)
imgs = []
for i in index:
    patch_id = type2[i]
    wsiname, ROI_id, gridx, gridy, patch_size = parse_patch_id(patch_id)
    im = wsi.read_region((int(gridx*patch_size), int(gridy*patch_size)), 0, (patch_size, patch_size)).convert('RGB')
    imgs.append(im)

plot_multiple(imgs, ['type1']*12, 3,4, (10, 5), 'tab20')

In [ ]:
# wsi_pts[10]
# wsi_pts[3] 打不开？？？

In [ ]:
wsi=openslide.OpenSlide(wsi_path)
img = wsi.read_region((0,0), wsi.level_count-1, wsi.level_dimensions[wsi.level_count-1]).convert('RGB')

scale_factor = 1 / float(patch_size)
print(f"scale_factor: {scale_factor}")

mask_im = generate_mask(wsi_path, anno_pt, scale_factor)

plot_oneline(img_list=[img, mask_im], caption_list=['original WSI', 'lobule type'], 
             figure_size=(8,8), save_pt=None) # blue: type1; green: type2; red: type3

In [ ]:
# get patch_ids within the annotations

# mask is scale_factor
# patch_id // scale_factor + patch_size // scale_factor // 2

df = pd.read_csv(csv_pt)
print(f"patient age: {np.unique(df['age'])}")

patch_ids = [i for i in df['patch_id']]
patch_ids

In [ ]:
# label patches as lobule type 1-3

In [ ]:
# export csv file